In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# 1. Read file

In [7]:
df = pd.read_csv('../Dataset/breast-cancer.csv')
df.head()

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

# 2. Find Null/NAN value

In [5]:
assert df.isna().sum().sum() == 0
assert df.isnull().sum().sum() == 0

# 3. Prepocessing

In [9]:
df = df.drop(columns='id')

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
